# Data clean

In [22]:
import os
csv_files = []
for root, dirs, files in os.walk('F:\dataset\csv\CICIoT2023'):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))
print(f"Total CSV files: {len(csv_files)}")

Total CSV files: 169


In [18]:
total_lines = 0
for file_path in csv_files:
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        total_lines += sum(1 for _ in f)
print(f"Total lines (including headers): {total_lines}")

Total lines (including headers): 46686748


In [19]:
output_dir = r'F:\dataset\merged csv'
os.makedirs(output_dir, exist_ok=True)

merged_file_path = os.path.join(output_dir, 'merged.csv')

# Corrected: robust header skipping logic
header_written = False

with open(merged_file_path, 'w', encoding='utf-8') as fout:
    for file_path in sorted(csv_files):
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as fin:
            lines = fin.readlines()
            if not header_written:
                fout.write(lines[0])  # write header once
                header_written = True
            fout.writelines(lines[1:])  # skip first line (header) in each file

print(f"✅ Merged file saved to: {merged_file_path}")

✅ Merged file saved to: F:\dataset\merged csv\merged.csv


In [82]:
import pandas as pd

df = pd.read_csv(r'F:\dataset\merged csv\merged.csv')

print(f"📏 Merged CSV has {df.shape[0]} rows and {df.shape[1]} columns.")

📏 Merged CSV has 46686579 rows and 47 columns.


In [83]:
print("Preview of the first few rows:")
print(df.head())

Preview of the first few rows:
   flow_duration  Header_Length  Protocol Type  Duration         Rate  \
0       0.000000          54.00           6.00     64.00     0.329807   
1       0.000000          57.04           6.33     64.00     4.290556   
2       0.000000           0.00           1.00     64.00    33.396799   
3       0.328175       76175.00          17.00     64.00  4642.133010   
4       0.117320         101.73           6.11     65.91     6.202211   

         Srate  Drate  fin_flag_number  syn_flag_number  rst_flag_number  ...  \
0     0.329807    0.0              1.0              0.0              1.0  ...   
1     4.290556    0.0              0.0              0.0              0.0  ...   
2    33.396799    0.0              0.0              0.0              0.0  ...   
3  4642.133010    0.0              0.0              0.0              0.0  ...   
4     6.202211    0.0              0.0              1.0              0.0  ...   

         Std  Tot size           IAT  Numbe

In [84]:
label_column = df.columns[46]
print(f"Label column name: {label_column}")

# Get counts of each unique label
label_counts = df[label_column].value_counts()

# Print results
print(f"Found {len(label_counts)} unique labels (attack types):\n")
for label, count in label_counts.items():
    print(f"{label:<30} --> {count:,} samples")

Label column name: label
Found 34 unique labels (attack types):

DDoS-ICMP_Flood                --> 7,200,504 samples
DDoS-UDP_Flood                 --> 5,412,287 samples
DDoS-TCP_Flood                 --> 4,497,667 samples
DDoS-PSHACK_Flood              --> 4,094,755 samples
DDoS-SYN_Flood                 --> 4,059,190 samples
DDoS-RSTFINFlood               --> 4,045,285 samples
DDoS-SynonymousIP_Flood        --> 3,598,138 samples
DoS-UDP_Flood                  --> 3,318,595 samples
DoS-TCP_Flood                  --> 2,671,445 samples
DoS-SYN_Flood                  --> 2,028,834 samples
BenignTraffic                  --> 1,098,195 samples
Mirai-greeth_flood             --> 991,866 samples
Mirai-udpplain                 --> 890,576 samples
Mirai-greip_flood              --> 751,682 samples
DDoS-ICMP_Fragmentation        --> 452,489 samples
MITM-ArpSpoofing               --> 307,593 samples
DDoS-UDP_Fragmentation         --> 286,925 samples
DDoS-ACK_Fragmentation         --> 285,104 sam

In [85]:
# Get label column (47th column, index 46)
label_column = df.columns[46]

# Filter only the required labels
filtered_df = df[df[label_column].isin(['DDoS-SlowLoris', 'BenignTraffic'])]

# Show result
print(f"Filtered dataset contains {filtered_df.shape[0]} rows and {filtered_df.shape[1]} columns.")
print(filtered_df[label_column].value_counts())

# Optional: Save filtered data
filtered_df.to_csv(r'F:\dataset\merged csv\cleaned_dataset.csv', index=False)
print("Filtered CSV saved to: F:\\dataset\\merged csv\\cleaned_datase.csv")

Filtered dataset contains 1121621 rows and 47 columns.
label
BenignTraffic     1098195
DDoS-SlowLoris      23426
Name: count, dtype: int64
Filtered CSV saved to: F:\dataset\merged csv\cleaned_datase.csv


In [86]:

slowloris_df = filtered_df[filtered_df[label_column] == 'DDoS-SlowLoris']
benign_df = filtered_df[filtered_df[label_column] == 'BenignTraffic']


benign_sampled_df = benign_df.sample(n=slowloris_df.shape[0], random_state=42)


balanced_df = pd.concat([slowloris_df, benign_sampled_df])


balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


print("\nBalanced dataset label counts:")
print(balanced_df[label_column].value_counts())


balanced_df.to_csv(r'F:\dataset\merged csv\dataset_balance.csv', index=False)
print("Balanced CSV saved to: F:\\dataset\\merged csv\\dataset_balance.csv")


Balanced dataset label counts:
label
DDoS-SlowLoris    23426
BenignTraffic     23426
Name: count, dtype: int64
Balanced CSV saved to: F:\dataset\merged csv\dataset_balance.csv
